# Natural Orbital Functional Theory: Dynamic (Scheduled) step size

Import libreries

In [1]:
import pynof
from scipy.linalg import eigh
import numpy as np
import csv
import sys
import time as t
# Define energy.py path, in our case is in myCode dir, two directories above
sys.path.insert(1, '../../myCode/')
# read files with optimization functions
from energy import *

Create molecule and choose basis set

In [2]:
#Molecules Definition
#Times given in Seconds, gotten on a Linux PC (12th Gen Intel(R) Core(TM) i9-12900K)
#with export OMP_NUM_THREADS=8
#Time for HNO3: 342.1247 s
#Time for O3: 87.6977 s
#Time for CHCL3: 945.1193 s

bf3 = pynof.molecule("""
0 1
 F     4.007177     0.908823     0.000000
 F     1.724922     0.908844     0.000000
 F     2.866001    -1.067658    -0.000000
 B     2.866001     0.249991    -0.000000
""")


hno3 = pynof.molecule("""
 0 1
    N       -0.151833    0.032380   -0.000040
    O       -1.021558   -0.782138    0.000011
    O        1.148151   -0.517360    0.000013
    O       -0.208610    1.237710    0.000014
    H        1.718969    0.267641   -0.000028
""")

chcl3 = pynof.molecule("""
  0 1
Cl        -0.09620       -1.67890        0.13940
Cl        -1.40590        0.92280        0.13940
Cl         1.50220        0.75610        0.13930
C         -0.00010        0.00000       -0.41810
H         -0.00010       -0.00010       -1.51110
""")
o3 = pynof.molecule("""
 0 1
 O     1.068900     0.000000     0.215300
 O    -0.024957     0.000000    -0.445578
 O    -1.108044     0.000000     0.232808
 """)

# Dictionary to facilitate molecule selection with 
# output file name

molecules = {
    "BF3": bf3,
    "HNO3": hno3,
    "O3": o3,
    "CHCL3": chcl3,
}

Run calculations with a dynamyc alpha value

In [ ]:
times = {}
#molecule_list = ["BF3","HNO3","O3"]
molecule_list = ["HNO3","O3","CHCL3"]


t_st=t.time()
for molecule in molecule_list:
    alpha_values = [0.002, 0.02,  0.08]   
    if molecule == "CHCL3":
        alpha_values = [0.02, 0.08]   

    st=t.time()

    #Define calulation
    mol = molecules[molecule]
    # Select basis set
    basis = "cc-pvdz"
    p = pynof.param(mol,basis)
    
    if molecule == "CHCL3":
        p.maxit = 15
    else: 
        p.maxit = 20

    p.orb_method="ADAM"

    # File name to store the data
    filename = f"{molecule}_{"NOF"}.csv"

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["alpha", "iteration", "energy"])  # Header

        for alpha in alpha_values:
            p.alpha = alpha
            # Select molecule
            _, energy_data = calc_nof_orbrot(mol, p)
            for i_ext, energy in energy_data:
                writer.writerow([alpha, i_ext, energy])

        # ADAM algorithm including dynamic modification, ADAM2
        p.orb_method="ADAM2"
        p.alpha = 0.02
        _, energy_data = calc_nof_orbrot(mol, p)
        for i_ext, energy in energy_data:
            writer.writerow([0.0, i_ext, energy])
        et=t.time()
        times[molecule]=et-st
t_et=t.time()

for molecule in molecule_list:
     print(f"Time for {molecule}: {times[molecule]:.4f} s")

print(f"Total Time:",t_et-t_st)

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry N          line   168 file /Users/pobmelat/miniconda3/envs/pynof00/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-4 entry O          line   198 file /Users/pobmelat/miniconda3/envs/pynof00/share/psi4/basis/cc-pvdz.gbs 
    atoms 5   entry H          line    22 file /Users/pobmelat/miniconda3/envs/pynof00/share/psi4/basis/cc-pvdz.gbs 

Number of basis functions                   (NBF)    = 61
Inactive Doubly occupied orbitals up too     (NO1)    = 0
No. considered Strongly Doubly occupied MOs (NDOC)   = 16
No. considered Strongly Singly occupied MOs (NSOC)   = 0
No. of Weakly occ. per St. Doubly occ.  MOs (NCWO)   = 2
Dimension of the Nat. Orb. subspace         (NBF5)   = 48
No. of electrons                                     = 32
Multiplicity                                         = 1

No violations of the orthonormality


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.



PNOF8 Calculation (ADAM/Softmax Optimization)

Nitext  Nit_orb  Nit_occ      Eelec           Etot          Ediff        Grad_orb   Grad_occ Conv Orb Conv Occ
alpha =  0.002
     0     30     82    -393.23222880  -266.76242143 -10392.23222880      1.3e+01    5.2e-02   False   True
alpha =  0.002
     1     30    144    -398.52981653  -272.06000916     -5.29758773      7.7e+00    1.9e-05   False   True
alpha =  0.002
     2     30     49    -401.68194386  -275.21213649     -3.15212732      5.5e+00    1.6e-05   False   True
alpha =  0.002
     3     30     45    -403.53057736  -277.06076999     -1.84863351      4.1e+00    1.5e-05   False   True
alpha =  0.002
     4     30     40    -404.73463027  -278.26482290     -1.20405291      3.0e+00    2.0e-05   False   True
alpha =  0.002
     5     30     36    -405.45999811  -278.99019074     -0.72536784      2.2e+00    1.4e-05   False   True
alpha =  0.002
     6     30     28    -405.89930430  -279.42949693     -0.43930620      1.7e+00    2.1